In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import requests
import bs4
import time
# import neo_quant

## open finder
!open .

## python file load
%load neo_quant.py

In [160]:
# %load neo_quant.py
import pandas as pd
import numpy as np

def make_code(x):
    x = str(x)
    return 'A' + '0' * (6-len(x)) + x


def make_code2(x):
    x = str(x)
    return '0' * (6-len(x)) + x


def get_all_company_info():
    kospi = pd.read_excel('data/kospi.xls')
    kosdaq = pd.read_excel('data/kosdaq.xls')
    kospi['구분'] = '코스피'
    kosdaq['구분'] = '코스닥'
    companies = pd.concat([kospi, kosdaq])
    companies = companies[['구분','종목코드','기업명']]
#     companies['종목코드'] = companies['종목코드'].apply(modifyCompanyCode)
    companies = companies.reset_index()
    return companies

def get_company_info(min_price):
    kospi = pd.read_excel('data/kospi.xls')
    kosdaq = pd.read_excel('data/kosdaq.xls')
    kospi['구분'] = '코스피'
    kosdaq['구분'] = '코스닥'
    companies = pd.concat([kospi, kosdaq])
    companies = companies[['종목코드','기업명','구분','액면가(원)','상장주식수(주)', '자본금(원)']]
    companies['상장주식수(주)'] = companies['상장주식수(주)'].str.replace(',','').astype(int)
    companies['자본금(원)'] = companies['자본금(원)'].str.replace(',','').astype(int)
    companies['액면가(원)'] = companies['액면가(원)'].str.replace(',','').astype(float)
    companies = companies[companies['액면가(원)'] >= min_price]
    companies['액면가(원)'] = companies['액면가(원)'].astype(int)
    companies = companies.set_index('종목코드')
    return companies

def company_apply_a_prefix(companies):
    old_index = pd.Series(companies.index)
    new_index = old_index.map(make_code)
    return companies.set_index(new_index)

def company_apply_no_prefix(companies):
    old_index = pd.Series(companies.index)
    new_index = old_index.map(make_code2)
    return companies.set_index(new_index)

# 액면가 1000원 이상 회사 리스트 가져와서 코드앞에 A 붙이기
# companies = get_company_info(1000)
# companies = apply_a_type_code(companies)

def filter_company(st_df, companies):
    new_company_list = []
    for num, code in enumerate(st_df.index):
        if code in companies.index:
            new_company_list.append(code)

    return st_df.loc[new_company_list]

def add_company_info(st_df, companies):
    return pd.merge(st_df, companies, how='inner', left_index=True, right_index=True)

# 데이터프레임에서 회사코드로 필터링한 후 회사 정보 추가하기
# roa_filter_info = add_company_info(filter_company(roa, companies), companies)


# [코드 3.15] 재무제표 데이터를 가져와 데이터프레임으로 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_fs_dataframe(firm_code):
    fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode=' + firm_code
    fs_page = requests.get(fs_url)
    fs_tables = pd.read_html(fs_page.text)
    
    temp_df = fs_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df[temp_df.columns[:4]]
    temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]

    temp_df2 = fs_tables[2]
    temp_df2 = temp_df2.set_index(temp_df2.columns[0])
    temp_df2 = temp_df2.loc[['자산', '부채', '자본']]

    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

    fs_df = pd.concat([temp_df, temp_df2, temp_df3])
    
    return fs_df

# [코드 3.19] 데이터프레임 형태 바꾸기 코드 함수화 (CH3. 데이터 수집하기.ipynb)

def change_df(firm_code, dataframe):
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code : dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]
        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)    
    
    return total_df

# [코드 3.21] 재무 비율 데이터프레임을 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_fr_dataframe(firm_code):
    fr_url = 'https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode=' + firm_code
    fr_page = requests.get(fr_url)
    fr_tables = pd.read_html(fr_page.text)
    
    temp_df = fr_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['유동비율계산에 참여한 계정 펼치기',
                           '부채비율계산에 참여한 계정 펼치기',
                           '영업이익증가율계산에 참여한 계정 펼치기',
                           'ROA계산에 참여한 계정 펼치기',
                           'ROIC계산에 참여한 계정 펼치기']]
    temp_df.index = ['유동비율', '부채비율', '영업이익률', 'ROA', 'ROIC']
    return temp_df

# [코드 3.23] 투자지표 데이터프레임을 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_invest_dataframe(firm_code):
    invest_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode=' + firm_code
    invest_page = requests.get(invest_url)
    invest_tables = pd.read_html(invest_page.text)
    temp_df = invest_tables[1]
    
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['PER계산에 참여한 계정 펼치기',
                       'PCR계산에 참여한 계정 펼치기',
                       'PSR계산에 참여한 계정 펼치기',
                       'PBR계산에 참여한 계정 펼치기',
                      '총현금흐름']]
    temp_df.index = ['PER', 'PCR', 'PSR', 'PBR', '총현금흐름']
    return temp_df

#  [코드 3.40] 가격을 가져와 데이터프레임 만드는 함수 (CH3. 데이터 수집하기 2.ipynb)

def make_price_dataframe(code, timeframe, count):
    url = 'https://fchart.stock.naver.com/sise.nhn?requestType=0'
    price_url = url + '&symbol=' + code + '&timeframe=' + timeframe + '&count=' + count
    price_data = requests.get(price_url)
    price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml')
    item_list = price_data_bs.find_all('item')
    
    date_list = []
    price_list = []
    for item in item_list:
        temp_data = item['data']
        datas = temp_data.split('|')
        date_list.append(datas[0])
        price_list.append(datas[4])

    price_df = pd.DataFrame({code:price_list}, index=date_list)
    
    return price_df

#  [코드 4.6] 재무 데이터 전처리하는 함수 (CH4. 전략 구현하기.ipynb)

def get_finance_data(path):
    data_path = path
    raw_data = pd.read_excel(data_path)
    raw_data = raw_data.set_index(raw_data.columns[0])
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])
    
    new_big_col = []
    for num, col in enumerate(big_col):
        if 'Unnamed' in col:
            new_big_col.append(new_big_col[num-1])
        else:
            new_big_col.append(big_col[num])
            
    raw_data.columns = [new_big_col, small_col]
    clean_df = raw_data.loc[ raw_data.index.dropna() ]
    
    return clean_df

def check_IFRS(x):
    if x == 'N/A(IFRS)':
        return np.NaN
    else:
        return x

#  [코드 4.14] PER기준으로 오름차순으로 정렬하여 주는 함수 (CH4. 전략 구현하기.ipynb)

def low_per(invest_df, index_date, num):
    invest_df[(index_date, 'PER')] = pd.to_numeric(invest_df[(index_date, 'PER')])
    per_sorted = invest_df.sort_values(by=(index_date, 'PER'))
    return per_sorted[index_date][:num]

#  [코드 4.15] ROA기준으로 내림차순으로 정렬하여 주는 함수 (CH4. 전략 구현하기.ipynb)

def high_roa(fr_df, index_date, num):
    fr_df[(index_date, 'ROA')] = fr_df[(index_date, 'ROA')].apply(check_IFRS)
    fr_df[(index_date, 'ROA')] = pd.to_numeric(fr_df[(index_date, 'ROA')] )
    sorted_roa = fr_df.sort_values(by=(index_date, 'ROA'), ascending=False)
    return sorted_roa[index_date][:num]


#  [코드 4.22] 마법공식 함수로 만들기 (CH4. 전략 구현하기.ipynb)

def magic_formula(fr_df, invest_df, index_date, num):
    per = low_per(invest_df, index_date, None)
    roa = high_roa(fr_df, index_date, None)
    per['per순위'] = per['PER'].rank()
    roa['roa순위'] = roa['ROA'].rank(ascending=False)
    magic = pd.merge(per, roa, how='outer', left_index=True, right_index=True)
    magic['마법공식 순위'] = (magic['per순위'] + magic['roa순위']).rank().sort_values()
    magic = magic.sort_values(by='마법공식 순위')
    return magic[:num]

#  [코드 4.23] 저평가 지수를 기준으로 정렬하여 순위 만들어 주는 함수 (CH4. 전략 구현하기.ipynb)

def get_value_rank(invest_df, value_type, index_date, num):
    invest_df[(index_date,  value_type)] = pd.to_numeric(invest_df[(index_date,  value_type)])
    value_sorted = invest_df.sort_values(by=(index_date,  value_type))[index_date]
    value_sorted[  value_type + '순위'] = value_sorted[value_type].rank()
    return value_sorted[[value_type, value_type + '순위']][:num]

#  [코드 4.25] 저평가 지표 조합 함수 (CH4. 전략 구현하기.ipynb)

def make_value_combo(value_list, invest_df, index_date, num):
    
    for i, value in enumerate(value_list):
        temp_df = get_value_rank(invest_df, value, index_date, None)
        if i == 0:
            value_combo_df = temp_df
            rank_combo = temp_df[value + '순위']
        else:
            value_combo_df = pd.merge(value_combo_df, temp_df, how='outer', left_index=True, right_index=True)
            rank_combo = rank_combo + temp_df[value + '순위']
    
    value_combo_df['종합순위'] = rank_combo.rank()
    value_combo_df = value_combo_df.sort_values(by='종합순위')
    
    return value_combo_df[:num]

#  [코드 4.29] F-score 함수(CH4. 전략 구현하기.ipynb)

def get_fscore(fs_df, index_date, num):
    fscore_df = fs_df[index_date]
    fscore_df['당기순이익점수'] = fscore_df['당기순이익'] > 0
    fscore_df['영업활동점수'] = fscore_df['영업활동으로인한현금흐름'] > 0
    fscore_df['더큰영업활동점수'] = fscore_df['영업활동으로인한현금흐름'] > fscore_df['당기순이익']
    fscore_df['종합점수'] = fscore_df[['당기순이익점수', '영업활동점수', '더큰영업활동점수']].sum(axis=1)
    fscore_df = fscore_df[fscore_df['종합점수'] == 3]
    return fscore_df[:num]

#  [코드 4.34] 모멘텀 데이터프레임 만들기 함수화 (CH4. 전략 구현하기.ipynb)

def get_momentum_rank(price_df, index_date, date_range, num):
    momentum_df = pd.DataFrame(price_df.pct_change(date_range).loc[index_date])
    momentum_df.columns = ['모멘텀']
    momentum_df['모멘텀순위'] = momentum_df['모멘텀'].rank(ascending=False)
    momentum_df = momentum_df.sort_values(by='모멘텀순위')
    return momentum_df[:num]

#  [코드 4.39] 저평가 + Fscore 함수화 (CH4. 전략 구현하기.ipynb)

def get_value_quality(invest_df, fs_df, index_date, num):
    value = make_value_combo(['PER', 'PBR', 'PSR', 'PCR'], invest_df, index_date, None)
    quality = get_fscore(fs_df, index_date, None)
    value_quality = pd.merge(value, quality, how='outer', left_index=True, right_index=True)
    value_quality_filtered = value_quality[value_quality['종합점수'] == 3]
    vq_df = value_quality_filtered.sort_values(by='종합순위')
    return vq_df[:num]

#  [코드 5.12] 백테스트 함수 버젼1 (Ch5. 백테스트.ipynb)


def backtest_beta(price_df, strategy_df, start_date, end_date, initial_money):

    code_list = []
    for code in strategy_df.index:
        code_list.append(code.replace('A',''))

    strategy_price = price_df[code_list][start_date:end_date]

    pf_stock_num = {}
    stock_amount = 0
    stock_pf = 0
    each_money = initial_money / len(strategy_df)
    for code in strategy_price.columns:
        temp = int( each_money / strategy_price[code][0] )
        pf_stock_num[code] = temp
        stock_amount = stock_amount + temp * strategy_price[code][0]
        stock_pf = stock_pf + strategy_price[code] * pf_stock_num[code]

    cash_amount = initial_money - stock_amount

    backtest_df = pd.DataFrame({'주식포트폴리오':stock_pf})
    backtest_df['현금포트폴리오'] = [cash_amount] * len(backtest_df)
    backtest_df['종합포트폴리오'] = backtest_df['주식포트폴리오'] + backtest_df['현금포트폴리오']
    backtest_df['일변화율'] = backtest_df['종합포트폴리오'].pct_change()
    backtest_df['총변화율'] = backtest_df['종합포트폴리오']/initial_money - 1
    
    return backtest_df

#  [코드 5.16] 해당 날짜에 가격이 없으면 투자 관련 데이터에서 해당 종목 없애는 함수 (Ch5. 백테스트.ipynb)

def select_code_by_price(price_df, data_df, start_date):
    new_code_list = []
    for code in price_df[start_date].iloc[0].dropna().index:
        new_code_list.append('A' + code)
        
    selected_df =  data_df.loc[new_code_list]
    return selected_df

# [코드 5.24] 백테스트 시작날짜가 주어지면 전략 기준 날짜를 계산하는 함수 (Ch5. 백테스트.ipynb)

def get_strategy_date(start_date):
    temp_year = int(start_date.split('-')[0])
    temp_month = start_date.split('-')[1]
    if temp_month in '1 2 3 4 5'.split(' '):
        strategy_date = str(temp_year - 2) + '/12'
    else:
        strategy_date = str(temp_year - 1) + '/12'
    return strategy_date


# [코드 5.32] 리밸런싱 백테스트 함수화 (Ch5. 백테스트.ipynb)

def backtest_re(strategy, start_date, end_date, initial_money, price_df, fr_df, fs_df, num, value_type=None, value_list=None, date_range=None):
    
    start_year = int(start_date.split('-')[0])
    end_year = int(end_date.split('-')[0])

    total_df = 0
    for temp in range(start_year, end_year):
        this_term_start = str(temp) + '-' + start_date.split('-')[1]
        this_term_end = str(temp+1) + '-' + start_date.split('-')[1]
        strategy_date = get_strategy_date(this_term_start)
        
        if strategy.__name__ == 'high_roa':
            st_df = strategy(select_code_by_price(price_df, fr_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'magic_formula':
            st_df = strategy(select_code_by_price(price_df, invest_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'get_value_rank':
            st_df = strategy(select_code_by_price(price_df, invest_df, this_term_start), value_type, strategy_date, num)
        elif strategy.__name__ == 'make_value_combo':
            st_df = strategy(value_list, select_code_by_price(price_df, invest_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'get_fscore':
            st_df = strategy(select_code_by_price(price_df, fs_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'get_momentum_rank':
            st_df = strategy(price_df, price_df[this_term_start].index[0] , date_range, num)
        elif strategy.__name__ == 'get_value_quality':
            st_df = strategy(select_code_by_price(price_df, invest_df, this_term_start), 
                             select_code_by_price(price_df, fs_df, this_term_start), strategy_date, num)
        
        backtest = backtest_beta(price_df, st_df, this_term_start, this_term_end, initial_money)
        temp_end = backtest[this_term_end].index[0]
        backtest = backtest[:temp_end]
        initial_money =  backtest['종합포트폴리오'][-1]
        if temp == start_year:
            total_df = backtest
        else:
            total_df = pd.concat([total_df[:-1], backtest])

    total_df ['일변화율'] = total_df ['종합포트폴리오'].pct_change()
    total_df ['총변화율'] = total_df ['종합포트폴리오']/ total_df ['종합포트폴리오'][0] - 1
    
    return total_df


# [코드 5.40] MDD 함수화 (Ch5. 백테스트.ipynb)

def get_mdd(back_test_df):
    max_list = [0]
    mdd_list = [0]

    for i in back_test_df.index[1:]:
        max_list.append(back_test_df['총변화율'][:i].max())
        if max_list[-1] > max_list[-2]:
            mdd_list.append(0)
        else:
            mdd_list.append(min(back_test_df['총변화율'][i] - max_list[-1], mdd_list[-1])   )

    back_test_df['max'] = max_list
    back_test_df['MDD'] = mdd_list
    
    return back_test_df




In [4]:
#  [코드 4.7] 재무 데이터 전처리하는 함수 (CH4. 전략 구현하기.ipynb)

fs_path = r'data/fs_data.xlsx'
fs_df = get_finance_data(fs_path)
fr_path = r'data/fr_data.xlsx'
fr_df = get_finance_data(fr_path)
invest_path = r'data/invest_data.xlsx'
invest_df = get_finance_data(invest_path)
price_path = r'data/price_data.xlsx'
price_df = pd.read_excel(price_path)
price_df = price_df.set_index(price_df.columns[0])

In [131]:
#  [코드 4.16] per, roa 정렬한 데이터프레임을 가져오기 (CH4. 전략 구현하기.ipynb)

date = '2015/12'
per = low_per(invest_df, date, None)
roa = high_roa(fr_df, date, None)

In [132]:
#  [코드 4.17] per 순위 만들기 (CH4. 전략 구현하기.ipynb)

per['per순위'] = per['PER'].rank()

In [133]:
#  [코드 4.18] roa 순위 만들기 (CH4. 전략 구현하기.ipynb)

roa['roa순위'] = roa['ROA'].rank(ascending=False)

In [134]:
magic = pd.merge(per, roa, how='outer', left_index=True, right_index=True)
magic['마법공식 순위'] = (magic['per순위'] + magic['roa순위']).rank()
magic = magic.sort_values(by='마법공식 순위')
magic = magic.dropna(subset=['마법공식 순위'])
magic

,PBR,PCR,PER,PSR,총현금흐름,per순위,ROA,ROIC,부채비율,영업이익률,유동비율,roa순위,마법공식 순위
Unnamed: 0,,,,,,,,,,,,,
A121440,0.56,0.31,0.31,1.64,17,1.0,113.9,-2.3,39.2,적지,93.8,2.0,1.5
A014470,0.79,0.49,0.50,1.11,100,2.0,115.8,5.4,16.5,90.8,213.1,1.0,1.5
A001520,0.68,1.05,1.06,1.69,364,4.0,60.3,11.0,23.4,45.3,485.4,5.0,3.0
A009180,1.31,1.75,1.84,0.28,29,10.0,41.8,-0.2,123.1,-75.3,126.4,9.0,4.0
A072870,0.63,1.54,1.69,0.76,355,8.0,26.0,19.3,23.2,176.0,374.1,36.0,5.0
A040350,1.53,4.29,4.32,5.65,-47,31.5,34.7,-30.7,13.5,적지,139.9,15.0,6.0
A003120,0.73,3.00,3.08,4.93,47,17.0,26.5,1.7,15.3,6.5,776.1,34.0,7.0
A033160,0.67,1.01,1.07,0.34,269,5.0,21.4,4.1,91.4,165.7,192.2,52.0,8.0
A036710,0.80,0.42,0.54,0.43,375,3.0,20.8,0.9,208.1,흑전,49.0,55.5,9.0


In [135]:
magic_filter = filter_company(magic, companies)

In [137]:
magic_filter = add_company_info(magic_filter, companies)

In [83]:
pd.options.display.max_rows = 60

In [ ]:
companies = get_company_info(1000)

In [90]:
companies = apply_a_type_code(companies)

In [ ]:
name_list = pd.Series(magic.index).apply(lambda x: companies.loc[x]['기업명'])
name_list
magic['기업명'] = name_list.values

In [147]:
value_combo_df = make_value_combo(['PER', 'PBR'], invest_df, '2018/12', 20)
value_combo_df = add_company_info(value_combo_df, companies)
value_combo_df

,PER,PER순위,PBR,PBR순위,종합순위,기업명,구분,액면가(원),상장주식수(주),자본금(원)
A003030,0.79,3.0,0.22,14.0,2.0,세아제강지주,코스피,5000,4141657,20708285000
A012320,3.23,33.0,0.22,14.0,3.0,경동인베스트,코스피,5000,2365023,11825115000
A003300,0.60,2.0,0.31,53.5,6.0,한일홀딩스,코스피,5000,5994356,31342450000
A009200,3.39,35.5,0.26,22.5,7.5,무림페이퍼,코스피,2500,41609310,104023275000
A003830,3.17,31.0,0.27,27.0,7.5,대한화섬,코스피,5000,1328000,6640000000
A016610,3.47,40.0,0.28,31.5,10.0,DB금융투자,코스피,5000,42446389,212231945000
A139130,3.66,44.0,0.32,62.5,12.0,DGB금융지주,코스피,5000,169145833,845729165000
A000860,4.21,66.0,0.31,53.5,13.0,강남제비스코,코스피,1000,6500000,6500000000
A006220,3.98,60.0,0.32,62.5,14.0,제주은행,코스피,5000,32128774,160643870000
A005010,5.06,112.5,0.21,12.0,15.0,휴스틸,코스피,5000,7621668,38108340000


In [149]:
value_combo_df = make_value_combo(['PER', 'PBR', 'PSR', 'PCR'], invest_df, '2018/12', 20)
value_combo_df = add_company_info(value_combo_df, companies)
value_combo_df

,PER,PER순위,PBR,PBR순위,PSR,PSR순위,PCR,PCR순위,종합순위,기업명,구분,액면가(원),상장주식수(주),자본금(원)
A003030,0.79,3.0,0.22,14.0,0.14,98.5,0.69,4.0,2.0,세아제강지주,코스피,5000,4141657,20708285000
A009200,3.39,35.5,0.26,22.5,0.11,67.0,1.08,18.0,3.0,무림페이퍼,코스피,2500,41609310,104023275000
A016710,5.24,127.0,0.25,18.0,0.11,67.0,1.82,34.0,5.0,대성홀딩스,코스피,1000,16089459,16089459000
A004960,1.20,8.0,0.44,200.5,0.09,46.5,1.19,20.5,6.0,한신공영,코스피,5000,11570702,57853510000
A004140,3.09,27.0,0.43,184.0,0.09,46.5,1.18,19.0,7.0,동방,코스피,1000,33465588,33465588000
A017940,4.90,98.0,0.30,46.0,0.08,32.5,3.14,128.5,8.0,E1,코스피,5000,6860000,34300000000
A002030,4.95,102.5,0.33,73.5,0.15,113.5,1.80,32.5,9.0,아세아,코스피,5000,2191024,10955120000
A003300,0.60,2.0,0.31,53.5,0.26,283.0,0.54,2.0,10.0,한일홀딩스,코스피,5000,5994356,31342450000
A013580,2.18,15.0,0.49,253.0,0.08,32.5,1.94,46.0,11.0,계룡건설산업,코스피,5000,8930907,44654535000
A034300,2.22,16.5,0.49,253.0,0.09,46.5,1.86,37.0,12.0,신세계건설,코스피,5000,4000000,20000000000


In [153]:
fscore_df = get_fscore(fs_df, '2018/12', 30)
fscore_df_filter = add_company_info(fscore_df, companies)
fscore_df_filter

/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,당기순이익,매출액,부채,영업이익,영업활동으로인한현금흐름,자본,자산,당기순이익점수,영업활동점수,더큰영업활동점수,종합점수,기업명,구분,액면가(원),상장주식수(주),자본금(원)
A006840,2066,37112,19240,2697,3173,14739,33979,True,True,True,3,AK홀딩스,코스피,5000,13247561,66237805000
A282330,1542,57759,9609,1895,3458,5217,14826,True,True,True,3,BGF리테일,코스피,1000,17283906,17283906000
A001460,91,1979,3310,213,491,4074,7384,True,True,True,3,BYC,코스피,5000,624615,4200000000
A001040,8800,295234,192077,13325,15411,125218,317295,True,True,True,3,CJ,코스피,5000,29176998,179047370000
A000120,518,92197,47378,2427,619,31389,78767,True,True,True,3,CJ대한통운,코스피,5000,22812344,114061720000
A037560,397,11780,8759,680,2626,10593,19351,True,True,True,3,CJ헬로,코스피,2500,77446865,193617162500
A016610,631,NaN,62823,864,1688,7609,70432,True,True,True,3,DB금융투자,코스피,5000,42446389,212231945000
A000990,868,6693,5081,1130,1803,5584,10665,True,True,True,3,DB하이텍,코스피,5000,44398588,222554520000
A017940,792,46302,22495,140,1847,13159,35654,True,True,True,3,E1,코스피,5000,6860000,34300000000
A078930,10305,177444,121060,22098,12692,105250,226310,True,True,True,3,GS,코스피,5000,92915378,473501020000


In [178]:
momentum_df = get_momentum_rank(price_df, '2019-10-1', 250, 200)
momentum_df_filter = company_apply_a_prefix(momentum_df)
momentum_df_filter
momentum_df_filter = add_company_info(momentum_df_filter, companies)
momentum_df_filter

,모멘텀,모멘텀순위,기업명,구분,액면가(원),상장주식수(주),자본금(원)
A002070,1.385387,24.0,남영비비안,코스피,1000,6867945,6867945000
A010280,0.871872,49.0,쌍용정보통신,코스닥,1000,40464034,40464034000
A084680,0.850440,53.0,이월드,코스피,1000,90509897,90509897000
A027970,0.844068,57.0,세하,코스피,1000,29562997,29562997000
A004800,0.776166,60.0,효성,코스피,5000,21071025,105355125000
A064960,0.671975,71.0,S&T모티브,코스피,5000,14623136,73115680000
A000080,0.653614,77.0,하이트진로,코스피,5000,70133611,368818745000
A159650,0.643478,80.0,하이골드8호,코스피,5000,14905929,74529645000
A001070,0.628692,83.0,대한방직,코스피,1000,5300000,5300000000
A012450,0.611321,86.0,한화에어로스페이스,코스피,5000,51560000,265650000000


In [179]:
#  [코드 4.35] 저평가 데이터프레임과 F-score 데이터프레임 만들기 (CH4. 전략 구현하기.ipynb)

date = '2018/12'
value = make_value_combo(['PER', 'PBR', 'PSR', 'PCR'], invest_df, date, None)
quality = get_fscore(fs_df, date, None)

/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:248: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:250: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [180]:
#  [코드 4.36] 저평가 데이터프레임과 F-score 데이터프레임 합치기(CH4. 전략 구현하기.ipynb)

value_quality = pd.merge(value, quality, how='outer', left_index=True, right_index=True)

In [182]:
#  [코드 4.37] 합친 데이터프레임에서 fscore가 3점인 종목만 남기기(CH4. 전략 구현하기.ipynb)

value_quality_filtered = value_quality[value_quality['종합점수'] == 3]

In [184]:
#  [코드 4.38] 남은 종목 저평가 순위로 정렬(CH4. 전략 구현하기.ipynb)

vq_df = value_quality_filtered.sort_values(by='종합순위')

In [186]:
vq_df_fiter = add_company_info(vq_df, companies)

In [192]:
vq_df_fiter[vq_df_fiter['구분']=='코스피']
vq_df_fiter[vq_df_fiter['액면가(원)'] >=5000]

,PER,PER순위,PBR,PBR순위,PSR,PSR순위,PCR,PCR순위,종합순위,당기순이익,...,자산,당기순이익점수,영업활동점수,더큰영업활동점수,종합점수,기업명,구분,액면가(원),상장주식수(주),자본금(원)
A017940,4.90,98.0,0.30,46.0,0.08,32.5,3.14,128.5,8.0,792,...,35654,True,True,True,3.0,E1,코스피,5000,6860000,34300000000
A002030,4.95,102.5,0.33,73.5,0.15,113.5,1.80,32.5,9.0,1002,...,28403,True,True,True,3.0,아세아,코스피,5000,2191024,10955120000
A013580,2.18,15.0,0.49,253.0,0.08,32.5,1.94,46.0,11.0,945,...,17262,True,True,True,3.0,계룡건설산업,코스피,5000,8930907,44654535000
A034300,2.22,16.5,0.49,253.0,0.09,46.5,1.86,37.0,12.0,431,...,7571,True,True,True,3.0,신세계건설,코스피,5000,4000000,20000000000
A005010,5.06,112.5,0.21,12.0,0.15,113.5,3.08,124.0,14.0,172,...,6035,True,True,True,3.0,휴스틸,코스피,5000,7621668,38108340000
A002200,4.50,80.0,0.34,85.5,0.28,317.5,2.64,96.5,32.0,173,...,2988,True,True,True,3.0,한국수출포장공업,코스피,5000,4000000,20000000000
A058650,11.88,542.0,0.20,10.0,0.08,32.5,1.57,31.0,35.0,645,...,54539,True,True,True,3.0,세아홀딩스,코스피,5000,4000000,20000000000
A004690,11.51,521.0,0.28,31.5,0.11,67.0,2.64,96.5,47.0,266,...,36994,True,True,True,3.0,삼천리,코스피,5000,4055025,20275125000
A001880,3.39,35.5,0.59,411.5,0.23,238.5,3.35,148.5,56.0,645,...,7961,True,True,True,3.0,삼호,코스피,5000,15179766,75898830000
A002310,3.57,42.5,0.53,314.0,0.36,457.5,2.44,81.5,65.0,787,...,8124,True,True,True,3.0,아세아제지,코스피,5000,8956502,44782510000


In [195]:
def get_kospi_list(st_df):
    return st_df[st_df['구분']=='코스피']
get_kospi_list(vq_df_fiter)
def get_price_over(st_df, price):
    return st_df[st_df['액면가(원)'] >= price]
get_price_over(vq_df_fiter, 5000)

,PER,PER순위,PBR,PBR순위,PSR,PSR순위,PCR,PCR순위,종합순위,당기순이익,...,자산,당기순이익점수,영업활동점수,더큰영업활동점수,종합점수,기업명,구분,액면가(원),상장주식수(주),자본금(원)
A017940,4.90,98.0,0.30,46.0,0.08,32.5,3.14,128.5,8.0,792,...,35654,True,True,True,3.0,E1,코스피,5000,6860000,34300000000
A002030,4.95,102.5,0.33,73.5,0.15,113.5,1.80,32.5,9.0,1002,...,28403,True,True,True,3.0,아세아,코스피,5000,2191024,10955120000
A013580,2.18,15.0,0.49,253.0,0.08,32.5,1.94,46.0,11.0,945,...,17262,True,True,True,3.0,계룡건설산업,코스피,5000,8930907,44654535000
A034300,2.22,16.5,0.49,253.0,0.09,46.5,1.86,37.0,12.0,431,...,7571,True,True,True,3.0,신세계건설,코스피,5000,4000000,20000000000
A005010,5.06,112.5,0.21,12.0,0.15,113.5,3.08,124.0,14.0,172,...,6035,True,True,True,3.0,휴스틸,코스피,5000,7621668,38108340000
A002200,4.50,80.0,0.34,85.5,0.28,317.5,2.64,96.5,32.0,173,...,2988,True,True,True,3.0,한국수출포장공업,코스피,5000,4000000,20000000000
A058650,11.88,542.0,0.20,10.0,0.08,32.5,1.57,31.0,35.0,645,...,54539,True,True,True,3.0,세아홀딩스,코스피,5000,4000000,20000000000
A004690,11.51,521.0,0.28,31.5,0.11,67.0,2.64,96.5,47.0,266,...,36994,True,True,True,3.0,삼천리,코스피,5000,4055025,20275125000
A001880,3.39,35.5,0.59,411.5,0.23,238.5,3.35,148.5,56.0,645,...,7961,True,True,True,3.0,삼호,코스피,5000,15179766,75898830000
A002310,3.57,42.5,0.53,314.0,0.36,457.5,2.44,81.5,65.0,787,...,8124,True,True,True,3.0,아세아제지,코스피,5000,8956502,44782510000


In [196]:
value_quality = get_value_quality(invest_df, fs_df, '2018/12', 20)

/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:248: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:250: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [198]:
value_quality
add_company_info(value_quality, companies)

,PER,PER순위,PBR,PBR순위,PSR,PSR순위,PCR,PCR순위,종합순위,당기순이익,...,자산,당기순이익점수,영업활동점수,더큰영업활동점수,종합점수,기업명,구분,액면가(원),상장주식수(주),자본금(원)
A009200,3.39,35.5,0.26,22.5,0.11,67.0,1.08,18.0,3.0,615,...,20143,True,True,True,3.0,무림페이퍼,코스피,2500,41609310,104023275000
A016710,5.24,127.0,0.25,18.0,0.11,67.0,1.82,34.0,5.0,238,...,11173,True,True,True,3.0,대성홀딩스,코스피,1000,16089459,16089459000
A004140,3.09,27.0,0.43,184.0,0.09,46.5,1.18,19.0,7.0,158,...,5541,True,True,True,3.0,동방,코스피,1000,33465588,33465588000
A017940,4.90,98.0,0.30,46.0,0.08,32.5,3.14,128.5,8.0,792,...,35654,True,True,True,3.0,E1,코스피,5000,6860000,34300000000
A002030,4.95,102.5,0.33,73.5,0.15,113.5,1.80,32.5,9.0,1002,...,28403,True,True,True,3.0,아세아,코스피,5000,2191024,10955120000
A013580,2.18,15.0,0.49,253.0,0.08,32.5,1.94,46.0,11.0,945,...,17262,True,True,True,3.0,계룡건설산업,코스피,5000,8930907,44654535000
A034300,2.22,16.5,0.49,253.0,0.09,46.5,1.86,37.0,12.0,431,...,7571,True,True,True,3.0,신세계건설,코스피,5000,4000000,20000000000
A005010,5.06,112.5,0.21,12.0,0.15,113.5,3.08,124.0,14.0,172,...,6035,True,True,True,3.0,휴스틸,코스피,5000,7621668,38108340000
A002710,4.57,82.0,0.39,133.5,0.11,67.0,2.96,113.0,16.0,98,...,3048,True,True,True,3.0,TCC스틸,코스피,1000,22342500,22342500000
A267290,4.59,84.0,0.49,253.0,0.10,58.5,2.71,99.0,24.0,342,...,7783,True,True,True,3.0,경동도시가스,코스피,2500,4717063,11792657500
